In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import trimesh
import glob
import os

In [ ]:
def get_obj_files(data_path, recursive=True):
    
    # Retrieve OBJ files from the data path
    obj_files = [file for file in glob.glob(data_path + r"\**\*.obj", recursive=recursive)]
    print(f"Found {len(obj_files)} OBJ files.")
    return obj_files

def save_npy_array_from_obj(obj_files, voxel_shape, voxel_resolution=0.025, save_npy=True, npy_filename='voxel_grids'):
    
    # Loop through the OBJ files and initialize an empty array that will contain np arrays
    print("Starting the pipeline...")
    np_arrays = []
    for i, obj_file in enumerate(obj_files):
    
        # Read the mesh from the OBJ file
        mesh = trimesh.load(obj_file, force='mesh')
        
        # Create a voxel grid
        voxelized = mesh.voxelized(pitch=voxel_resolution)
        voxelized = voxelized.revoxelized(shape=voxel_shape)
        voxel_matrix = voxelized.matrix
        
        # Reshape the array to (voxel_shape, 1)
        new_shape = tuple(list(voxel_shape) + [1])
        voxel_matrix = voxel_matrix.reshape(new_shape)
        np_arrays.append(voxel_matrix)
        print(f"{i+1}", end=" ")
    
    # Save the voxel matrix as an NPY file
    np_arrays_stacked = np.stack(tuple(np_arrays), axis=0)
    print(f"Final NumPy array shape: {np_arrays_stacked.shape}")

    # Save the NumPy array as an NPY file if specified, otherwise, return the array
    if save_npy:
        np.save(npy_filename + ".npy", np_arrays_stacked)
        return None
    return np_arrays_stacked

# Save the paths of each of the OBJ files into a list
DATA_PATH = r"C:\Users\aanal\Documents\sem3\nureal_network_and_deep_learning\Project\Data"
obj_files = get_obj_files(DATA_PATH)

In [ ]:
# Convert OBJ files into voxel grids, and save them into a Numpy array
save_npy_array_from_obj(obj_files, (32, 32, 32), voxel_resolution=0.025)

In [ ]:
def get_category_names(data_path):

    # Initialize an empty array and loop through the folders within data path
    category_names = []
    for folder in os.scandir(data_path):

        # if the folder name starts with a number then it's a category name
        folder_basename = os.path.basename(folder.path)
        if folder.is_dir() and folder_basename[0].isdigit():
            category_names.append(folder_basename)
    return category_names

categories = get_category_names(r"C:\Users\aanal\Documents\sem3\nureal_network_and_deep_learning\Project\Data")
print(f"We have {len(categories)} categories.")
print(categories)

In [ ]:
def save_npy_array_from_obj_per_category(data_path, voxel_shape, voxel_resolution=0.025):

    # Get the category names (55)
    categories = get_category_names(data_path)
    print(f"Found {len(categories)} categories.")

    # Loop through the categories
    for i, category in enumerate(categories):
        print(f"================ Starting process for category #{i+1}, name: {category} ================")

        # Set the category data path and retrieve all obj files
        category_path = data_path + '\\' + category
        obj_files = get_obj_files(category_path, recursive=True)
        
        # Convert OBJ files into voxel grids, and save them into a Numpy array
        save_npy_array_from_obj(obj_files, voxel_shape, voxel_resolution, save_npy=True, npy_filename=category+"_voxels")

data_path = r"C:\Users\aanal\Documents\sem3\nureal_network_and_deep_learning\Project\Data"
save_npy_array_from_obj_per_category(data_path, (32, 32, 32))

In [ ]:
def render_images_from_obj_files(obj_files, renderer_git_path, n_views, output_path, randomize_view, start_id=0):
    
    # Save current path and change to the stanford-shapenet-renderer tool path
    current_path = os.getcwd()
    os.chdir(renderer_git_path)
    
    # For each OBJ file, generate the images using the renderer tool
    print("Starting the pipeline (generating images)...")
    for i, obj_file in enumerate(obj_files):
        print(f"{i+1}", end=" ")
        command_str = f'blender --background --python render_blender.py -- --views {n_views} --id_render {start_id+i+1} --randomize_view {int(randomize_view)} --output_folder {output_path} {obj_file}'
        os_return = os.system(command_str)
        
        # Check if process was executed succesfully
        if os_return != 0:
            print(f"Error... os returned: {os_return}")
            break
            
    # Return to the original path
    os.chdir(current_path)

def render_images_per_category(categories, n_views, renderer_path, data_path, output_path, randomize_view=False):
    print(f"Found {len(categories)} categories.")
    
    # Loop through each of the categories
    for i, category in enumerate(categories):
        print(f"\n================ Starting process for category #{i+1}, name: {category} ================")

        # Set the category data path and retrieve all obj files
        category_path = data_path + '\\' + category
        obj_files = get_obj_files(category_path, recursive=True)
        
        # Render the images according to the parameters
        render_images_from_obj_files(obj_files, renderer_path, n_views, output_path + "/" + category, randomize_view)

data_path = r"C:\Users\aanal\Documents\sem3\nureal_network_and_deep_learning\Project\Data"
renderer_path = r"C:\Users\aanal\Documents\sem3\nureal_network_and_deep_learning\Project\stanford-shapenet-renderer"
output_path = "/Users/aanal/Documents/sem3/nureal_network_and_deep_learning/Project/op/"
render_images_per_category(categories, 10, renderer_path, data_path, output_path, randomize_view=False)